In [11]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('~/dsi/Submissions/Projects/project-5-group-project-master/kiva_numeric_models.csv')

In [10]:
df.head(2)

,ORIGINAL_LANGUAGE,DESCRIPTION_TRANSLATED,LOAN_AMOUNT,STATUS,IMAGE_ID,VIDEO_ID,ACTIVITY_NAME,SECTOR_NAME,LOAN_USE,COUNTRY_CODE,...,DISTRIBUTION_MODEL,word_count_DT,word_count_LU,word_count_tags,char_count_DT,char_count_LU,char_count_tags,FEM_COUNT,MALE_COUNT,month
0,Spanish,The city of Portoviejo is located in the valle...,1075.0,1,1,0,Natural Medicines,Health,to purchase natural products.,EC,...,field_partner,192,4,5,924,26,36,1,1,1
1,English,"Lorna is a married woman, 39 years old with fi...",400.0,0,1,0,Personal Products Sales,Retail,"to purchase additional stocks of Avon, Natasha...",PH,...,field_partner,75,12,3,371,69,28,1,1,3


In [7]:
#Converting long text columns to length of words and count of characters
#Step1 is to drop nulls
df = df.drop(df[(df['DESCRIPTION_TRANSLATED'].isnull()==True)].index)
df = df.drop(df[(df['TAGS'].isnull()==True)].index)
df = df.drop(df[(df['LOAN_USE'].isnull()==True)].index)


#Step 2 is to update new columns
def word_len_count(column):
    word_count = len(column.split())
    return word_count

df['word_count_DT'] = df['DESCRIPTION_TRANSLATED'].apply(word_len_count)
df['word_count_TAGS'] = df['TAGS'].apply(word_len_count)
df['word_count_LU'] = df['LOAN_USE'].apply(word_len_count)

def char_len_count(column):
    char_count = column.replace(' ','')
    char_count = len(char_count[:])
    return char_count

df['char_count_DT'] = df['DESCRIPTION_TRANSLATED'].apply(char_len_count)
df['char_count_TAGS'] = df['TAGS'].apply(char_len_count)
df['char_count_LU'] = df['LOAN_USE'].apply(char_len_count)

In [8]:
df.head(2)

,LOAN_ID,LOAN_NAME,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,FUNDED_AMOUNT,LOAN_AMOUNT,STATUS,IMAGE_ID,VIDEO_ID,...,BORROWER_PICTURED,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL,year,word_count_DT,word_count_TAGS,word_count_LU,char_count_DT,char_count_TAGS,char_count_LU
0,1455352,Raisa Jokasta,Spanish,"En la ciudad de Portoviejo, conocida como la c...",The city of Portoviejo is located in the valle...,1075.0,1075.0,1,2638561.0,NaN,...,true,monthly,field_partner,2018,192,5,4,924,36,26
1,1727469,Lorna,English,"Lorna is a married woman, 39 years old with fi...","Lorna is a married woman, 39 years old with fi...",225.0,400.0,0,3108106.0,NaN,...,true,monthly,field_partner,2019,75,3,12,371,28,69


In [12]:
#Mark whether Image and Video was provided
df['IMAGE_ID'] = np.where(df['IMAGE_ID'].isnull(), 0, 1)
df['VIDEO_ID'] = np.where(df['VIDEO_ID'].isnull(), 0, 1)

## Parse out borrower genders & borrower pictured
df['FEM_COUNT'] = df['BORROWER_GENDERS'].str.count("female")
df['MALE_COUNT'] = df['BORROWER_GENDERS'].str.count("male")
df['PIC_TRUE_COUNT'] = df['BORROWER_PICTURED'].str.count("true")
df['PIC_FALSE_COUNT'] = df['BORROWER_PICTURED'].str.count("false")


df['ANY_FEM'] = df['BORROWER_GENDERS'].str.contains('female').map({True:1, False:0})
df['ANY_MALE'] = df['BORROWER_GENDERS'].str.contains('male').map({True:1, False:0})
df['month'] = df['POSTED_TIME'].str[6:7]


#Drop unrequired columns to run numeric models

df.drop(columns = ['DESCRIPTION_TRANSLATED', 'LOAN_USE', 'TAGS', 'DESCRIPTION','LOAN_ID','LOAN_NAME', 'FUNDED_AMOUNT',
                     'DESCRIPTION', 'TOWN_NAME', 'CURRENCY_EXCHANGE_COVERAGE_RATE', 'CURRENCY', 'PARTNER_ID', 'PLANNED_EXPIRATION_TIME',
                     'DISBURSE_TIME', 'POSTED_TIME', 'RAISED_TIME', 'NUM_LENDERS_TOTAL', 'NUM_JOURNAL_ENTRIES', 'NUM_BULK_ENTRIES',
                     'BORROWER_NAMES', 'COUNTRY_NAME', 'CURRENCY_POLICY','year', 'BORROWER_GENDERS', 'BORROWER_PICTURED', 'IMAGE_ID',
                     'VIDEO_ID'], inplace=True)

In [13]:
df.to_csv('kivasmall_sr.csv', index=False)